In [1]:
%cd /workspace/flair
# !python -m spacy download en_core_web_lg

/workspace/flair


# abcnews

In [164]:
import numpy as np
import pandas as pd

df = pd.read_csv("/workspace/flair/data/abcnews-date-text.csv")
df = df.head(1000)

In [165]:
from flair.embeddings import Sentence, WordEmbeddings, FlairEmbeddings, DocumentPoolEmbeddings, DocumentRNNEmbeddings

document_embeddings = DocumentPoolEmbeddings([WordEmbeddings('glove')])

X = []
for i, d in enumerate(df["headline_text"]):
    sent = Sentence(d, use_tokenizer=True)
    document_embeddings.embed(sent)
    x = sent.get_embedding()
    x = x.detach().numpy()
    X.append(x)
X = np.stack(X)

In [206]:
# df.groupby(by="publish_date").agg(["count"])
groups = df.groupby(by="publish_date")
groups.groups.keys()
keys = [20030219, 20030220, 20030221, 20030222, 20030223, 20030224]
# groups[20030219]
# groups.get_group(20030219)
d0 = groups.get_group(keys[0])
d1 = groups.get_group(keys[1])
d2 = groups.get_group(keys[1])

# for i, d in enumerate(df["headline_text"]):
#     print(i, d)
#     break

X0 = X[d0.index[0]:d0.index[-1]+1]
X1 = X[d1.index[0]:d1.index[-1]+1]
# X2 = X[d2.index[0]:d2.index[-1]+1]

# X0_1 = X[d0.index[0]:d1.index[-1]+1]

In [227]:
from sklearn.cluster import KMeans, DBSCAN, AffinityPropagation, MeanShift, AgglomerativeClustering

clustering = KMeans(n_clusters=50, random_state=0).fit(X0)
# clustering.predict(X1)
y = np.concatenate([c0.labels_, clustering.predict(X1)])
# c1 = KMeans(n_clusters=45, random_state=0).fit(X1)

# # z = np.zeros(df.shape[0], dtype=np.int32)
# z = np.empty((df.shape[0],)) * np.nan
# z[:c0.labels_.shape[0]] = c0.labels_
# df["c0"] = z

# z = np.empty((df.shape[0],)) * np.nan
# z[:c1.labels_.shape[0]] = c1.labels_
# df["c1"] = z
# df.head(100)

_df = df[0:len(y)]
_df["y"] = y
# _df.loc[_df["y"] == 20, ["headline_text", "y"]]
_df.groupby("y").agg(["count"])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,publish_date,headline_text,c0,c1
,count,count,count,count
y,,,,
0,4,4,3,3
1,37,37,15,20
2,10,10,9,9
3,38,38,8,15
4,14,14,14,14
5,14,14,8,9
6,6,6,5,6
7,22,22,15,15


In [205]:
gs = df.groupby("c0")
df.loc[df["c1"] == 2, ["headline_text", "c0", "c1"]]

,headline_text,c0,c1
22,call for ethanol blend fuel to go ahead,18.0,2.0
69,health minister backs organ and tissue storage,5.0,2.0
85,last minute call hands alinghi big lead,22.0,2.0
100,more women urged to become councillors,20.0,2.0
117,omodei to stay in politics,28.0,2.0
145,rain eases wheatbelt water woes,17.0,2.0
184,uni to continue tree disease study,20.0,2.0
189,vowles to retire at end of season,28.0,2.0
215,brisbane sparkies head for 10 day strike,NaN,2.0
219,bungle leaves doctor waiting to practise,NaN,2.0


In [189]:
a = np.array([1,2,3])
z = np.zeros(5, dtype=np.int16)
z[:a.shape[0]] = a
# z
# a.shape
# a.shape[0]
z

array([1, 2, 3, 0, 0], dtype=int16)

In [87]:
from sklearn.cluster import KMeans, DBSCAN, AffinityPropagation, MeanShift, AgglomerativeClustering

# clustering = KMeans(n_clusters=100, random_state=0).fit(X)
# clustering = DBSCAN(eps=0.11, min_samples=1, metric="cosine").fit(X)
# clustering = AffinityPropagation(max_iter=2000, convergence_iter=15,).fit(X)
# clustering = MeanShift(max_iter=3000).fit(X)
# clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=3).fit(X)

print(clustering.labels_)
df["y"] = list(clustering.labels_)

# print(kmeans.predict([[0, 0], [12, 3]]))
# print(kmeans.cluster_centers_)
# df["label"] = kmeans.labels_
# df[df["label"] == 0]

# len(df)
df.loc[df["y"] == 1, ["headline_text", "y"]]

[ 82   0 120  91  91  65  26 109  21 115 212 148 234 225  30 107  73 105
   3  75   0 153  49 148 213  36  40 132 114 260  15 229 229 159  33  90
   9 109 135  93 159  47  23 168  42 219  76  76 156  24 170  84  84 120
 254 254 205  70  24 193  45 159 151 154  15 212  12 115  79  36  48 169
 214  90 161   9 120  52 136  62 205  60 185 130 168 100  91  52   8 203
   8 166 176 121 244  76 120 171 152  30 132 260  73 150 135 115 183 203
  87  25 213  32  39 190  10 207 240 135 122 132 181  41  19 106  19 217
 126 226 259 120 128 154  24 203  80  18 235  16   6  91 234  82   6  81
 145 129  61 178 223  47 237 111 206 118 119  55  56  81 100 137  64  90
 155  12  62 171  45  34  49  16 165 152  41 219  28  41 122  22  15  70
 106 204 124 132  25 105  16 260 139  29  26 215 227  58 186 201 141  14
 257 100 162 114 176 170 175  79  96  82 124 234 157  13 148  96  93  29
  64  10 141 256 117 183 120  27 252 101 201  67 251  40  46  19  46 200
   0 131  81  97 114  76  41 118 160  51  77 189  9

,headline_text,y
735,former star koen licks cats,1
842,cat lovers march for romes sacred strays,1
844,cats claw dogs pies swoop on blues,1
866,fake crocs scare off flamingos,1
945,tourists cry foul over spilt milk,1


# Pandas

In [159]:
import pandas as pd

df = pd.DataFrame({"a" : [1,2,3,4,5],"b" : [6, 7, 8, 9,10]})
# df["d"] = None
# df.loc[0, "d"] = 1
# df["a"].tolist()
# df.loc[0:2, "c"] = [0,1]
# df["c"] = None
# df.loc[0:2, "c"] = [0,1,2]
df["c"] = np.zeros(5).astype(np.int16)
df
# df["c"].apply(pd.to_numeric)
a = np.zeros(5).astype(np.int16)
a[0:3] = np.array([1,2,3])
a

# groups = df.groupby(by="publish_date")
# groups.iloc['20030219']
# .agg(["count"])
# for name, group in groups:
#     print(name)
#     for i, row in group.iterrows():
#         print(row['headline_text'])
#         col = row['column']
#         column_type = row['column_type']
#         is_null = 'NOT NULL' if row['is_null'] == 'NO' else ''
#         print('\t{} {} {},'.format(col, column_type, is_null))

# gs

array([1, 2, 3, 0, 0], dtype=int16)

# ???

In [34]:
from pprint import pprint
import os 
import json 
directory = "/workspace/stocknet-dataset/tweet/raw"

for ticker in os.listdir(directory):
    p = os.path.join(directory, ticker)
    if os.path.isfile(p):
        continue
    for fn in os.listdir(p):
        with open(os.path.join(directory, ticker, fn)) as f:
            for i, l in enumerate(f.readlines()):
                print(i)
                j = json.loads(l)
#                 print(j["text"])
                pprint(j)
#                 print(l)
        break
    break

# [f for f in listdir(p)]
# for root, dirnames, filenames in os.walk(d):
#     for fileanme in filenames:
#         p = os.path.join(root, dirname, filename)
#         print(p)
#         with open(p) as f:
#             pass
#         break
        
#     for filename in filenames:
#         if ".DS_Store" in filename:
#             continue
#         path = os.path.join(root, filename)
#         print(root, filename)
#         break


0
{'contributors': None,
 'coordinates': None,
 'created_at': 'Tue Oct 14 13:39:49 +0000 2014',
 'entities': {'hashtags': [],
              'symbols': [{'indices': [0, 4], 'text': 'GDP'},
                          {'indices': [42, 47], 'text': 'TSLA'},
                          {'indices': [48, 53], 'text': 'TWTR'},
                          {'indices': [54, 56], 'text': 'C'},
                          {'indices': [57, 61], 'text': 'GDP'},
                          {'indices': [62, 67], 'text': 'AMZN'},
                          {'indices': [68, 73], 'text': 'DRYS'},
                          {'indices': [74, 77], 'text': 'VZ'}],
              'trends': [],
              'urls': [{'display_url': 'twitter.com/CBOE/status/52…',
                        'expanded_url': 'https://twitter.com/CBOE/status/522018942626594816',
                        'indices': [81, 104],
                        'url': 'https://t.co/7Q6GX3gUuS'},
                       {'display_url': 'quantpost.com/list/?symbo

In [1]:
import pandas as pd
from gensim.parsing.preprocessing import remove_stopwords

df = pd.read_csv("/workspace/flair/data/Full-Economic-News-DFE-839861.csv", encoding='ISO-8859-1')
# documents = df.loc[:100, "text"].tolist()
df = df.loc[:10, ["headline"]]
df['label'] = [i for i in range(0,11)]
df

,headline,label
0,Yields on CDs Fell in the Latest Week,0
1,The Morning Brief: White House Seeks to Limit ...,1
2,Banking Bill Negotiators Set Compromise --- Pl...,2
3,Manager's Journal: Sniffing Out Drug Abusers I...,3
4,Currency Trading: Dollar Remains in Tight Rang...,4
5,"Stocks Fall Again; BofA, Alcoa Slide",5
6,"Bank of Montreal, Royal Bank Profits Rose in 2...",6
7,Battle Over Medical Costs Isn't Over,7
8,Sequenom to Buy Gemini Genomics In Stock Accord,8
9,U.S. Dollar Falls Against Most Currencies; Dec...,9


In [55]:
# documents
df = pd.DataFrame({"d" : ["aaa" ,"bbb", "ccc"]})
df['label'] = [1,1,0]
df[df.label == 1]

[x for x in df["d"].head(1)]

['aaa']

# flair

In [59]:
from flair.data import Sentence
from flair.models import SequenceTagger

# make a sentence
sentence = Sentence('I love Berlin .')

# load the NER tagger
tagger = SequenceTagger.load('ner')

# run NER over sentence
tagger.predict(sentence)

2020-03-30 15:11:26,618 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/models-v0.4/NER-conll03-english/en-ner-conll03-v0.4.pt not found in cache, downloading to /tmp/tmpo8_dribe


100%|██████████| 432197603/432197603 [02:45<00:00, 2604231.44B/s]

2020-03-30 15:14:14,169 copying /tmp/tmpo8_dribe to cache at /root/.flair/models/en-ner-conll03-v0.4.pt


2020-03-30 15:14:15,801 removing temp file /tmp/tmpo8_dribe
2020-03-30 15:14:15,961 loading file /root/.flair/models/en-ner-conll03-v0.4.pt


[Sentence: "I love Berlin ."   [− Tokens: 4  − Token-Labels: "I love Berlin <S-LOC> ."]]

In [109]:
from sklearn.cluster import KMeans
import numpy as np
X = np.array([[1, 2], [1, 4], [1, 0], [10, 2], [10, 4], [10, 0]])
# X.shape
# kmeans = KMeans(n_clusters=2, random_state=0).fit(X)
# print(kmeans.labels_)
# print(kmeans.predict([[0, 0], [12, 3]]))
# print(kmeans.cluster_centers_)

(6, 2)

In [79]:
import pandas as pd

df = pd.read_csv("/workspace/flair/data/Full-Economic-News-DFE-839861.csv", encoding='ISO-8859-1')
df = df.head(100)
# [d for d in df["headline"]]
# df.columns
g = df.groupby(["date"])
g.get_group('11/23/11')
# s = g.size()
# s[s > 1]

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,positivity,positivity:confidence,relevance,relevance:confidence,articleid,date,headline,positivity_gold,relevance_gold,text
5,842613460,False,finalized,3,12/4/15 23:15,3.0,0.6783,yes,1.0,wsj_905654974,11/23/11,"Stocks Fall Again; BofA, Alcoa Slide",NaN,NaN,"Stocks declined, as investors weighed slower-t..."
56,842613511,False,finalized,3,12/5/15 7:26,NaN,NaN,no,1.0,wsj_905646207,11/23/11,Indian Rupee Falls Again,NaN,NaN,MUMBAI - The Indian rupee fell against the U.S...


In [14]:
import pandas as pd

df = pd.read_csv("/workspace/flair/data/abcnews-date-text.csv")
df = df.head(2000)

In [15]:
g = df.groupby(["publish_date"])
g.size()



publish_date
20030219    198
20030220    250
20030221    250
20030222    126
20030223    136
20030224    250
20030225    250
20030226    250
20030227    221
20030228     69
dtype: int64

In [58]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from flair.embeddings import Sentence, WordEmbeddings, FlairEmbeddings, DocumentPoolEmbeddings, DocumentRNNEmbeddings

df = pd.read_csv("/workspace/flair/data/Full-Economic-News-DFE-839861.csv", encoding='ISO-8859-1')
df = df.head(100)

document_embeddings = DocumentPoolEmbeddings([WordEmbeddings('glove')])

X = []
for d in df["headline"]:
    sent = Sentence(d, use_tokenizer=True)
    document_embeddings.embed(sent)
    x = sent.get_embedding()
    x = x.detach().numpy()
#     print(x)
    X.append(x)

X = np.stack(X)
kmeans = KMeans(n_clusters=10, random_state=0).fit(X)
# print(len(kmeans.labels_))
df["y"] = list(kmeans.labels_)
# print(kmeans.predict([[0, 0], [12, 3]]))
# print(kmeans.cluster_centers_)
# df["label"] = kmeans.labels_
# df[df["label"] == 0]

# len(df)
df.loc[df["y"] == 1, ["headline", "y"]]

In [64]:
X = np.stack(X)
kmeans = KMeans(n_clusters=10, random_state=0).fit(X)
# print(len(kmeans.labels_))
df["y"] = list(kmeans.labels_)
# print(kmeans.predict([[0, 0], [12, 3]]))
# print(kmeans.cluster_centers_)
# df["label"] = kmeans.labels_
# df[df["label"] == 0]

# len(df)
df.loc[df["y"] == 1, ["headline", "y"]]

,headline,y
4,Currency Trading: Dollar Remains in Tight Rang...,1
9,U.S. Dollar Falls Against Most Currencies; Dec...,1
17,Dollar Declines as Players Take Profits From R...,1
26,Bond Prices Tumble as Dollar's Plunge Prompts ...,1
35,"Tuesday's markets: Prices of bonds, stocks dro...",1
47,Retailers Stock Up on Caution,1
48,Some Stocks Had Big Fourth-Quarter Gains Despi...,1
52,Bond Prices Are Little Changed In Slow Trading...,1
60,Futures Markets: Treasury Bond Contracts End N...,1
62,"AMR, Southwest Profits Surge Even as Fuel Pric...",1


[7, 9, 1, 0, 2, 8, 3, 5, 6, 2, 4]

In [97]:
# from flair.data import Sentence
from flair.embeddings import Sentence, WordEmbeddings, FlairEmbeddings, DocumentPoolEmbeddings, DocumentRNNEmbeddings

sentence = Sentence('this is one. this is two. this is three.', use_tokenizer=True)
# sentence.tokens
# now check out the embedded tokens.
# for token in sentence:
#     print(token)
#     print(token.embedding)

# document_embeddings = DocumentPoolEmbeddings([WordEmbeddings('glove'),
#                                               FlairEmbeddings('news-backward'),
#                                               FlairEmbeddings('news-forward')])
document_embeddings = DocumentPoolEmbeddings([WordEmbeddings('glove')])
document_embeddings.embed(sentence)
# print(sentence.get_embedding())

# from flair.embeddings import WordEmbeddings, DocumentRNNEmbeddings

# glove_embedding = WordEmbeddings('glove')
# document_embeddings = DocumentRNNEmbeddings([glove_embedding])
# sentence = Sentence('The grass is green . And the sky is blue .')
# document_embeddings.embed(sentence)
# print(sentence.get_embedding())

# document_lstm_embeddings = DocumentRNNEmbeddings([WordEmbeddings('glove')], rnn_type='LSTM')
# document_embeddings.embed(sentence)
# print(sentence.get_embedding())

x = sentence.get_embedding()
x.detach().numpy()

(100,)

In [61]:
from flair.data import Sentence

# sent = Sentence('The grass is green.', use_tokenizer=True)
sent = Sentence('The grass is green.')

In [62]:
from flair.data import segtok_tokenizer, Sentence
from flair.datasets import ClassificationDataset
from flair.datasets import ClassificationCorpus
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentPoolEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer

# ds = ClassificationDataset(
#     path_to_file='/workspace/flair/data/news/train.txt',
#     in_memory=True,
# )
# ds[0]

# corpus = ClassificationCorpus(data_folder='/workspace/flair/data/news', tokenizer=segtok_tokenizer, in_memory=True)
# corpus.train[0]
# label_dict = corpus.make_label_dictionary()

# document_embeddings = DocumentPoolEmbeddings([WordEmbeddings('glove')])
word_embeddings = [WordEmbeddings('glove'),
                   # comment in flair embeddings for state-of-the-art results
                   # FlairEmbeddings('news-forward'),
                   # FlairEmbeddings('news-backward'),
                   ]
document_embeddings = DocumentPoolEmbeddings([WordEmbeddings('glove'),
                   FlairEmbeddings('news-forward'),
                   FlairEmbeddings('news-backward'),])

document_embeddings(sent)

NotImplementedError: 

In [ ]:
document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)

classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)
trainer = ModelTrainer(classifier, corpus)
trainer.train('resources/taggers/ag_news',
              learning_rate=0.1,
              mini_batch_size=32,
              anneal_factor=0.5,
              patience=5)

# from flair.visual.training_curves import Plotter
# plotter = Plotter()
# plotter.plot_weights('resources/taggers/ag_news/weights.txt')


# classifier = TextClassifier.load('resources/taggers/ag_news/final-model.pt')
# sentence = Sentence('France is the current world cup winner.')
# classifier.predict(sentence)
# print(sentence.labels)

In [10]:
from flair.data import Sentence
from flair.embeddings import WordEmbeddings

sent = Sentence('The grass is green.', use_tokenizer=True)
# glove = WordEmbeddings('glove')
# glove.embed(sent)

# # now check out the embedded tokens.
# for token in sentence:
#     print(token)
#     print(token.embedding)

# [x.embedding.shape for x in sent]
# [x for x in sent]
sent

Sentence: "The grass is green ."   [− Tokens: 5]

# gensim

In [41]:
import pandas as pd
from gensim.parsing.preprocessing import remove_stopwords

df = pd.read_csv("/workspace/flair/data/Full-Economic-News-DFE-839861.csv", encoding='ISO-8859-1')
documents = df.loc[:1000, "text"].tolist()
# docs[0]
# documents = [
#     "Human machine interface for lab abc computer applications",
#     "A survey of user opinion of computer system response time",
#     "The EPS user interface management system",
#     "System and human system engineering testing of EPS",
#     "Relation of user perceived response time to error measurement",
#     "The generation of random binary unordered trees",
#     "The intersection graph of paths in trees",
#     "Graph minors IV Widths of trees and well quasi ordering",
#     "Graph minors A survey",
# ]

from pprint import pprint  # pretty-printer
from collections import defaultdict

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in remove_stopwords(d).lower().split()]
    for d in documents
]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

# pprint(texts)

from gensim import corpora

dictionary = corpora.Dictionary(texts)
# dictionary.save('/tmp/deerwester.dict')  # store the dictionary, for future reference
# print(dictionary)
corpus = [dictionary.doc2bow(text) for text in texts]
# corpus

In [44]:
from gensim import models

tfidf = models.TfidfModel(corpus)  # step 1 -- initialize a model
corpus_tfidf = tfidf[corpus]
# for doc in corpus_tfidf:
#     print(doc)

# lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=3)  # initialize an LSI transformation
model = models.LdaModel(corpus, id2word=dictionary, num_topics=10)
corpus_lsi = model[corpus_tfidf]  # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi
model.print_topics(10)

# both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
# for doc, as_text in zip(corpus_lsi, documents):
#     print(doc, as_text)


[(0,
  '0.011*"--" + 0.009*"the" + 0.007*"u.s." + 0.006*"said" + 0.005*"federal" + 0.004*"mr." + 0.004*"market" + 0.004*"fed" + 0.004*"economic" + 0.003*"new"'),
 (1,
  '0.011*"the" + 0.009*"new" + 0.007*"market" + 0.007*"said" + 0.007*"u.s." + 0.005*"mr." + 0.005*"stock" + 0.005*"fed" + 0.005*"economic" + 0.005*"--"'),
 (2,
  '0.008*"--" + 0.007*"new" + 0.007*"u.s." + 0.005*"market" + 0.005*"said" + 0.005*"the" + 0.004*"stock" + 0.003*"economic" + 0.003*"billion" + 0.003*"rates"'),
 (3,
  '0.009*"the" + 0.007*"new" + 0.006*"stock" + 0.006*"u.s." + 0.006*"prices" + 0.005*"said" + 0.004*"market" + 0.004*"economic" + 0.004*"--" + 0.004*"companies"'),
 (4,
  '0.011*"--" + 0.009*"the" + 0.007*"said" + 0.007*"new" + 0.007*"u.s." + 0.006*"market" + 0.006*"stock" + 0.005*"economic" + 0.004*"rates" + 0.004*"york"'),
 (5,
  '0.010*"the" + 0.009*"u.s." + 0.008*"--" + 0.006*"said" + 0.006*"stock" + 0.006*"investors" + 0.005*"market" + 0.005*"new" + 0.005*"year" + 0.004*"rates"'),
 (6,
  '0.009*"s

In [18]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_lg")  # make sure to use larger model!

dog dog 1.0
dog cat 0.80168545
dog banana 0.24327646
cat dog 0.80168545
cat cat 1.0
cat banana 0.2815437
banana dog 0.24327646
banana cat 0.2815437
banana banana 1.0


In [27]:
# F,Ford Motor Company,NYQ,Auto Manufacturers - Major,USA,,,
# MSFT,Microsoft Corporation,NMS,Business Software & Services,USA,,,
tks = nlp("AXP American Express Company")
# t1, t2 = nlp("F Ford")

# for token in tks:
#     print(token.text, token.has_vector, token.vector_norm, token.is_oov)

# for token1 in tks:
token1 = tks[0]
for token2 in tks[1:]:
    print(token1.text, token2.text, token1.similarity(token2))

AXP American 0.041551307
AXP Express 0.011548235
AXP Company -0.17779963


In [40]:
t1 = "The Walt Disney Company"
t2 = "The Walt Disney"
d1 = nlp(t1)
d2 = nlp(t2)

# type(doc.ents[0])
# displacy.render(doc, style="ent")
# s = doc.ents[0]
# s.vector
# d1.similarity(d2)


The Walt Disney Company


In [2]:
import spacy


t = 'NEW YORK -- Yields on most certificates of deposit offered by major banks dropped more than a tenth of a percentage point in the latest week, reflecting the overall decline in short-term interest rates. On small-denomination, or "consumer," CDs sold directly by banks, the average yield on six-month deposits fell to 5.49% from 5.62% in the week ended yesterday, according to an 18-bank survey by Banxquote Money Markets, a Wilmington, Del., information service.</br></br>On three-month "consumer" deposits, the average yield sank to 5.29% from 5.42% the week before, according to Banxquote. Two banks in the Banxquote survey, Citibank in New York and CoreStates in Pennsylvania, are paying less than 5% on threemonth small-denomination CDs.</br></br>Declines were somewhat smaller on five-year consumer CDs, which eased to 7.37% from 7.45%, Banxquote said.</br></br>Yields on three-month and six-month Treasury bills sold at Monday\'s auction plummeted more than a fifth of a percentage point from the previous week, to 5.46% and 5.63%, respectively.'

nlp = spacy.load("en_core_web_sm")
doc = nlp(t)

In [6]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)
displacy.render(doc, style="ent")
doc[0]
doc[0].text, doc[0].ent_type_, doc[0].ent_kb_id_

In [1]:
from flair.data import Corpus
from flair.datasets import TREC_6
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer

corpus = TREC_6()
label_dict = corpus.make_label_dictionary()
word_embeddings = [WordEmbeddings('glove'),
                   # comment in flair embeddings for state-of-the-art results
                   FlairEmbeddings('news-forward'),
                   # FlairEmbeddings('news-backward'),
                   ]
document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)
trainer = ModelTrainer(classifier, corpus)

trainer.train('resources/taggers/ag_news',
              learning_rate=0.1,
              mini_batch_size=32,
              anneal_factor=0.5,
              patience=5,
              max_epochs=150)

from flair.visual.training_curves import Plotter
plotter = Plotter()
plotter.plot_weights('resources/taggers/ag_news/weights.txt')

2020-03-11 14:01:10,984 Reading data from /root/.flair/datasets/trec_6
2020-03-11 14:01:10,987 Train: /root/.flair/datasets/trec_6/train.txt
2020-03-11 14:01:11,001 Dev: None
2020-03-11 14:01:11,007 Test: /root/.flair/datasets/trec_6/test.txt
2020-03-11 14:01:11,864 Computing label dictionary. Progress:


100%|██████████| 4907/4907 [00:00<00:00, 21493.21it/s]

2020-03-11 14:01:12,101 [b'LOC', b'ENTY', b'HUM', b'NUM', b'ABBR', b'DESC']


ValueError: cannot reshape array of size 417659 into shape (400000,100)

In [ ]:
from flair.models import TextClassifier
from flair.data import Sentence

classifier = TextClassifier.load('en-sentiment')
sentence = Sentence('Flair is pretty neat!')

classifier.predict(sentence)
# print sentence with predicted labels

print('Sentence above is: ', sentence.labels)

In [4]:
from flair.data import Sentence
from flair.models import SequenceTagger

# t = 'NEW YORK -- Yields on most certificates of deposit offered by major banks dropped more than a tenth of a percentage point in the latest week, reflecting the overall decline in short-term interest rates.</br></br>On small-denomination, or "consumer," CDs sold directly by banks, the average yield on six-month deposits fell to 5.49% from 5.62% in the week ended yesterday, according to an 18-bank survey by Banxquote Money Markets, a Wilmington, Del., information service.</br></br>On three-month "consumer" deposits, the average yield sank to 5.29% from 5.42% the week before, according to Banxquote. Two banks in the Banxquote survey, Citibank in New York and CoreStates in Pennsylvania, are paying less than 5% on threemonth small-denomination CDs.</br></br>Declines were somewhat smaller on five-year consumer CDs, which eased to 7.37% from 7.45%, Banxquote said.</br></br>Yields on three-month and six-month Treasury bills sold at Monday\'s auction plummeted more than a fifth of a percentage point from the previous week, to 5.46% and 5.63%, respectively.'
t = "iTV Will Boost Apple http:\/\/t.co\/8dup4cQc08 $AAPL #APPLE"
t = t.replace("AAPL", "")

sentence = Sentence(t)
# [x for x in sentence]

tagger = SequenceTagger.load("chunk-fast")
# tagger = SequenceTagger.load("ner-ontonotes-fast")
tagger.predict(sentence)

# print("Analysing %s" % sentence)
print(sentence.to_tagged_string())

2020-03-11 13:28:36,083 loading file /root/.flair/models/en-chunk-conll2000-fast-v0.4.pt
iTV <S-NP> Will <B-VP> Boost <E-VP> Apple <B-NP> http:\/\/t.co\/8dup4cQc08 <I-NP> $ <I-NP> #APPLE <E-NP>


In [5]:
sentence.get_spans('np')

[<NP-span (1): "iTV">,
 <VP-span (2,3): "Will Boost">,
 <NP-span (4,5,6,7): "Apple http:\/\/t.co\/8dup4cQc08 $ #APPLE">]

In [4]:
from flair.data import Sentence
from flair.models import SequenceTagger

t = 'NEW YORK -- Yields on most certificates of deposit offered by major banks dropped more than a tenth of a percentage point in the latest week, reflecting the overall decline in short-term interest rates.</br></br>On small-denomination, or "consumer," CDs sold directly by banks, the average yield on six-month deposits fell to 5.49% from 5.62% in the week ended yesterday, according to an 18-bank survey by Banxquote Money Markets, a Wilmington, Del., information service.</br></br>On three-month "consumer" deposits, the average yield sank to 5.29% from 5.42% the week before, according to Banxquote. Two banks in the Banxquote survey, Citibank in New York and CoreStates in Pennsylvania, are paying less than 5% on threemonth small-denomination CDs.</br></br>Declines were somewhat smaller on five-year consumer CDs, which eased to 7.37% from 7.45%, Banxquote said.</br></br>Yields on three-month and six-month Treasury bills sold at Monday\'s auction plummeted more than a fifth of a percentage point from the previous week, to 5.46% and 5.63%, respectively.'

tagger = SequenceTagger.load("chunk-fast")
sentence = Sentence(t)
tagger.predict(sentence)
sentence.to_tagged_string()

2020-02-28 06:10:34,080 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/models-v0.4/release-chunk-fast-0/en-chunk-conll2000-fast-v0.4.pt not found in cache, downloading to /tmp/tmpdhmzfc8v


100%|██████████| 75233247/75233247 [00:37<00:00, 2018218.86B/s]

2020-02-28 06:11:13,724 copying /tmp/tmpdhmzfc8v to cache at /root/.flair/models/en-chunk-conll2000-fast-v0.4.pt


2020-02-28 06:11:13,899 removing temp file /tmp/tmpdhmzfc8v
2020-02-28 06:11:13,930 loading file /root/.flair/models/en-chunk-conll2000-fast-v0.4.pt


'NEW <B-NP> YORK <E-NP> -- Yields <S-NP> on <S-PP> most <B-NP> certificates <E-NP> of <S-PP> deposit <S-NP> offered <S-VP> by <S-PP> major <B-NP> banks <E-NP> dropped <S-VP> more <B-NP> than <I-NP> a <I-NP> tenth <E-NP> of <S-PP> a <B-NP> percentage <I-NP> point <E-NP> in <S-PP> the <B-NP> latest <I-NP> week, <E-NP> reflecting <S-VP> the <B-NP> overall <I-NP> decline <E-NP> in <S-PP> short-term <B-NP> interest <I-NP> rates.</br></br>On <E-NP> small-denomination, or "consumer," <B-NP> CDs <E-NP> sold <S-VP> directly <S-ADVP> by <S-PP> banks, <S-NP> the <B-NP> average <I-NP> yield <E-NP> on <S-PP> six-month <B-NP> deposits <E-NP> fell <S-VP> to <S-PP> 5.49% <S-NP> from <S-PP> 5.62% <S-NP> in <S-PP> the <B-NP> week <E-NP> ended <S-VP> yesterday, <S-NP> according <S-PP> to <S-PP> an <B-NP> 18-bank <I-NP> survey <E-NP> by <S-PP> Banxquote <B-NP> Money <I-NP> Markets, <E-NP> a <B-NP> Wilmington, <I-NP> Del., <I-NP> information <I-NP> service.</br></br>On <I-NP> three-month <I-NP> "consumer" 

In [5]:
for entity in sentence.get_spans('ner'):
    print(entity)

In [4]:
import pandas as pd

df = pd.read_csv("/workspace/flair/data/Full-Economic-News-DFE-839861.csv", encoding='ISO-8859-1')
# df.head
# _unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,positivity,positivity:confidence,relevance,relevance:confidence,articleid,date,headline,positivity_gold,relevance_gold,text
# a, b, c = df.loc[0, ["date", "headline", "text"]]
# df.loc[12, ["date", "headline", "text"]][1]

'Dow Falls 45.95, Late GM Surge Stanches Losses'

In [23]:
# df.loc[10, ["date", "headline", "text"]][2]
docs = df.loc[:10, "text"].tolist()
docs[0]

'NEW YORK -- Yields on most certificates of deposit offered by major banks dropped more than a tenth of a percentage point in the latest week, reflecting the overall decline in short-term interest rates.</br></br>On small-denomination, or "consumer," CDs sold directly by banks, the average yield on six-month deposits fell to 5.49% from 5.62% in the week ended yesterday, according to an 18-bank survey by Banxquote Money Markets, a Wilmington, Del., information service.</br></br>On three-month "consumer" deposits, the average yield sank to 5.29% from 5.42% the week before, according to Banxquote. Two banks in the Banxquote survey, Citibank in New York and CoreStates in Pennsylvania, are paying less than 5% on threemonth small-denomination CDs.</br></br>Declines were somewhat smaller on five-year consumer CDs, which eased to 7.37% from 7.45%, Banxquote said.</br></br>Yields on three-month and six-month Treasury bills sold at Monday\'s auction plummeted more than a fifth of a percentage po

In [2]:
from flair.data import Sentence
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger.load("ner")

sentence: Sentence = Sentence("George Washington went to Washington .")
tagger.predict(sentence)

print("Analysing %s" % sentence)
print("\nThe following NER tags are found: \n")
print(sentence.to_tagged_string())

2020-02-26 13:30:40,873 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/models-v0.4/NER-conll03-english/en-ner-conll03-v0.4.pt not found in cache, downloading to /tmp/tmp4xhqochp


100%|██████████| 432197603/432197603 [47:36<00:00, 151296.68B/s] 

2020-02-26 14:18:18,476 copying /tmp/tmp4xhqochp to cache at /root/.flair/models/en-ner-conll03-v0.4.pt


2020-02-26 14:18:19,110 removing temp file /tmp/tmp4xhqochp
2020-02-26 14:18:19,240 loading file /root/.flair/models/en-ner-conll03-v0.4.pt
Analysing Sentence: "George Washington went to Washington ." - 6 Tokens

The following NER tags are found: 

George <B-PER> Washington <E-PER> went to Washington <S-LOC> .
